In [3]:
import pandas as pd

Import clean data

In [4]:
good_bad = pd.read_csv("good_bad_clients.csv")
application_records = pd.read_csv("application_record_clean.csv")
credit_records = pd.read_csv("credit_record_clean.csv")

In [20]:
def month_weight(month, number_of_records, mnor, historical_range):
    
    """
    Funkcja przyporządkowująca wagę danemu miesiącu (month) z uwzględnieniem liczby wszystkich miesięcy (number_of_months).
    Wagi maleją w sposób liniowy, a ich suma daje 1.
    """
    
    k = abs(month) + 1
    n = number_of_records
    
    if n < mnor:
        weight = 0
    elif mnor <= n <= historical_range:
        weight = -2*k/(n*(n+1))+2/n
    else:
        n = historical_range
        if k <= historical_range:
            weight = -2*k/(n*(n+1))+2/n
        else:
            weight = 0
              
                
    return weight

In [18]:
def good_bad_function(ID, status, threshold):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

In [11]:
def account_activity(ID, last_record):
    
    """
    Funkcja zwracająca obecny stan konta klienta.
    Konto aktywne - 1
    Konto nieaktywne - 0
    """
        
    if last_record == 0:
        active = 1
    else:
        active = 0
        
        
    return active

In [19]:
def status2number(month, status, number_of_records, mnor, historical_range):
    
    """
    W oparciu o funkcję month_weight, status2number przekształca status z wybranego miesiąca na liczbę z przedziału <-1,1>.
    
    """
    
    if month <= historical_range-1:
        weight = month_weight(month,
                              number_of_records,
                              mnor,
                              historical_range)

        if status is "5":
            value = -1
        elif status is "X":
            value = 0
        elif status is "C":
            value = 1
        else:
            n = int(status)
            value = 1/2**(n+1)-1
    else:
        weight = 1
        value = 0
    
    
    return weight * value

<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-19-9579a0827f27>:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if status is "5":
<ipython-input-19-9579a0827f27>:16: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "X":
<ipython-input-19-9579a0827f27>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "C":


In [14]:
def good_bad_function(ID, status, threshold):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= threshold:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

Function to create date frame with selecterd threshold

In [16]:
def main(data_frame, minimum_number_of_records=4, historical_range=24, threshold=0):
    
    """
    text
    text
    
    """
    
    data_frame["MONTHS_BALANCE"] = data_frame["MONTHS_BALANCE"].abs()
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].min().reset_index(name="LAST_RECORD")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    extra_frame = data_frame.groupby(by="ID")["MONTHS_BALANCE"].count().reset_index(name="NUMBER_OF_RECORDS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["MONTHS"] = data_frame["MONTHS_BALANCE"] - data_frame["LAST_RECORD"]
    

    data_frame["ACTIVE"] = data_frame.apply(lambda df: account_activity(df["ID"],
                                                                        df["LAST_RECORD"]), axis=1)
    
    data_frame["PARTIAL_STATUS"] = data_frame.apply(lambda df: status2number(df["MONTHS"],
                                                                             df["STATUS"],
                                                                             df["NUMBER_OF_RECORDS"],
                                                                             minimum_number_of_records,
                                                                             historical_range), axis=1)
    
    extra_frame = data_frame.groupby(by = "ID")["PARTIAL_STATUS"].sum().reset_index(name="TOTAL_STATUS")
    data_frame = pd.merge(data_frame, extra_frame, on="ID", how="left")
    
    data_frame["GOOD_BAD"] = data_frame.apply(lambda df: good_bad_function(df["ID"],
                                                                           df["TOTAL_STATUS"],
                                                                           threshold), axis=1)
    
    return data_frame

In [24]:
credit_records_new_df = main(credit_records, threshold = 0.4)

In [37]:
good_bad_dict_df = credit_records_new_df[["ID","GOOD_BAD"]].drop_duplicates().reset_index(drop = True)


In [38]:
good_bad_dict_df

,ID,GOOD_BAD
0,5001711,0
1,5001712,1
2,5001713,0
3,5001714,0
4,5001715,0
...,...,...
45980,5150482,0
45981,5150483,0
45982,5150484,0
45983,5150485,0


Join data frame good bad client with client data

In [41]:
final_df = pd.merge(application_records, good_bad_dict_df, how="left", on = "ID")

In [48]:
final_df[["ID", "CODE_GENDER"]]

,ID,CODE_GENDER
0,5008804,M
1,5008805,M
2,5008806,M
3,5008808,F
4,5008809,F
...,...,...
438552,6840104,M
438553,6840222,F
438554,6841878,F
438555,6842765,F
